# Lab 4.5.2: Agent Playground

**Module:** 4.5 - Demo Building & Prototyping  
**Time:** 3 hours  
**Difficulty:** ⭐⭐⭐⭐☆

---

## 🎯 Lab Objectives

Build a multi-page Streamlit application that visualizes AI agent reasoning:
- [ ] Multi-page architecture with shared state
- [ ] Agent chat interface with tool call visualization
- [ ] Thinking process display (chain-of-thought)
- [ ] Tool configuration and testing panel
- [ ] Session history and analytics
- [ ] Proper caching for model loading
- [ ] Deployment to Streamlit Cloud

---

## 🌍 Scenario

Your team has built an AI agent with multiple tools (calculator, web search, code executor). For internal demos and debugging, you need a playground where developers can:
1. Chat with the agent
2. See exactly what the agent is thinking
3. Watch tool calls happen in real-time
4. Configure which tools are available

Let's build it!

---

## Part 1: Project Setup

First, let's create the multi-page Streamlit app structure.

In [ ]:
# Install dependencies
!pip install -q streamlit>=1.30.0

In [ ]:
import os

# Create app directory structure
app_dir = '/tmp/agent_playground'
os.makedirs(f'{app_dir}/pages', exist_ok=True)
os.makedirs(f'{app_dir}/.streamlit', exist_ok=True)
os.makedirs(f'{app_dir}/utils', exist_ok=True)

print("Directory structure created:")
print(f"  {app_dir}/")
print(f"  ├── pages/")
print(f"  ├── utils/")
print(f"  └── .streamlit/")

## Part 2: Shared Utilities

Create shared utilities that all pages can use.

In [ ]:
# utils/__init__.py
init_py = '''"""Shared utilities for Agent Playground."""

from .agent import MockAgent, AVAILABLE_TOOLS
from .state import init_session_state, get_session_stats
'''

with open(f'{app_dir}/utils/__init__.py', 'w') as f:
    f.write(init_py)

print("Created utils/__init__.py")

In [ ]:
# utils/agent.py - Mock agent implementation
agent_py = '''"""Mock Agent for demonstration purposes."""

import time
import json
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, asdict
from datetime import datetime
import math

# Available tools definition
AVAILABLE_TOOLS = {
    "calculator": {
        "name": "Calculator",
        "description": "Perform mathematical calculations",
        "icon": "🧮",
        "enabled_by_default": True
    },
    "web_search": {
        "name": "Web Search",
        "description": "Search the web for information",
        "icon": "🔍",
        "enabled_by_default": True
    },
    "code_executor": {
        "name": "Code Executor",
        "description": "Execute Python code snippets",
        "icon": "💻",
        "enabled_by_default": False
    },
    "weather": {
        "name": "Weather",
        "description": "Get current weather information",
        "icon": "🌤️",
        "enabled_by_default": True
    }
}

@dataclass
class ToolCall:
    """Represents a tool call made by the agent."""
    tool_name: str
    input_args: Dict[str, Any]
    output: Any
    duration_ms: float
    success: bool

@dataclass
class AgentResponse:
    """Complete response from the agent."""
    content: str
    thinking: str
    tool_calls: List[ToolCall]
    total_duration_ms: float
    timestamp: str


class MockAgent:
    """
    Mock AI Agent for demonstration.
    
    In production, replace with your actual agent implementation.
    """
    
    def __init__(self, enabled_tools: Optional[List[str]] = None):
        self.enabled_tools = enabled_tools or ["calculator", "web_search", "weather"]
    
    def _execute_tool(self, tool_name: str, args: Dict) -> ToolCall:
        """Execute a tool and return results."""
        start_time = time.time()
        success = True
        output = None
        
        try:
            if tool_name == "calculator":
                expression = args.get("expression", "0")
                # Safe eval for basic math
                output = eval(expression, {"__builtins__": {}}, {"math": math})
            
            elif tool_name == "web_search":
                query = args.get("query", "")
                time.sleep(0.5)  # Simulate API call
                output = [
                    {"title": f"Result 1 for {query}", "url": "https://example.com/1"},
                    {"title": f"Result 2 for {query}", "url": "https://example.com/2"},
                ]
            
            elif tool_name == "code_executor":
                code = args.get("code", "")
                # In production, use a proper sandbox!
                output = f"Executed code: {code[:50]}..."
            
            elif tool_name == "weather":
                location = args.get("location", "Unknown")
                time.sleep(0.3)  # Simulate API call
                output = {
                    "location": location,
                    "temperature": "72°F",
                    "condition": "Sunny",
                    "humidity": "45%"
                }
            
            else:
                output = f"Unknown tool: {tool_name}"
                success = False
                
        except Exception as e:
            output = f"Error: {str(e)}"
            success = False
        
        duration = (time.time() - start_time) * 1000
        
        return ToolCall(
            tool_name=tool_name,
            input_args=args,
            output=output,
            duration_ms=round(duration, 2),
            success=success
        )
    
    def _generate_thinking(self, message: str) -> str:
        """Generate mock thinking/reasoning process."""
        thinking = f"""Analyzing user request: "{message}"

Step 1: Understanding the request
- The user is asking about: {message[:50]}...
- This requires me to analyze the content and determine the best approach.

Step 2: Determining tools needed
- Available tools: {\', \'.join(self.enabled_tools)}
"""
        # Detect tool needs based on keywords
        if any(word in message.lower() for word in ["calculate", "math", "sum", "+", "-", "*", "/"]):
            thinking += "- Detected mathematical query → Will use Calculator tool\n"
        if any(word in message.lower() for word in ["search", "find", "look up", "what is"]):
            thinking += "- Detected search query → Will use Web Search tool\n"
        if any(word in message.lower() for word in ["weather", "temperature", "forecast"]):
            thinking += "- Detected weather query → Will use Weather tool\n"
        
        thinking += "\nStep 3: Formulating response\n"
        thinking += "- Gathering information from tools...\n"
        thinking += "- Synthesizing response...\n"
        
        return thinking
    
    def respond(self, message: str) -> AgentResponse:
        """Generate a complete response to the user message."""
        start_time = time.time()
        
        # Generate thinking
        thinking = self._generate_thinking(message)
        
        # Determine and execute tools
        tool_calls = []
        message_lower = message.lower()
        
        if "calculator" in self.enabled_tools and any(
            word in message_lower for word in ["calculate", "math", "sum"]
        ):
            # Try to extract expression
            tool_calls.append(self._execute_tool("calculator", {"expression": "2 + 2"}))
        
        if "web_search" in self.enabled_tools and any(
            word in message_lower for word in ["search", "find", "what is"]
        ):
            tool_calls.append(self._execute_tool("web_search", {"query": message[:50]}))
        
        if "weather" in self.enabled_tools and any(
            word in message_lower for word in ["weather", "temperature"]
        ):
            tool_calls.append(self._execute_tool("weather", {"location": "San Francisco"}))
        
        # Generate response
        content = self._generate_response(message, tool_calls)
        
        total_duration = (time.time() - start_time) * 1000
        
        return AgentResponse(
            content=content,
            thinking=thinking,
            tool_calls=[asdict(tc) for tc in tool_calls],
            total_duration_ms=round(total_duration, 2),
            timestamp=datetime.now().isoformat()
        )
    
    def _generate_response(self, message: str, tool_calls: List[ToolCall]) -> str:
        """Generate the final response based on tool outputs."""
        response = f"I\'ve processed your request: \'{message[:100]}\'\n\n"
        
        if tool_calls:
            response += "Based on my analysis:\n\n"
            for tc in tool_calls:
                tool_info = AVAILABLE_TOOLS.get(tc.tool_name, {})
                response += f"{tool_info.get('icon', '🔧')} **{tc.tool_name}**: "
                if isinstance(tc.output, dict):
                    response += json.dumps(tc.output, indent=2)
                else:
                    response += str(tc.output)
                response += "\n\n"
        else:
            response += "I\'ve analyzed your request and formulated a response based on my knowledge.\n\n"
            response += "*Note: No tools were needed for this query.*"
        
        return response
'''

with open(f'{app_dir}/utils/agent.py', 'w') as f:
    f.write(agent_py)

print("Created utils/agent.py")

In [ ]:
# utils/state.py - Session state helpers
state_py = '''"""Session state management utilities."""

import streamlit as st
from datetime import datetime
from typing import Dict, Any

def init_session_state():
    """Initialize all session state variables."""
    
    # Messages history
    if "messages" not in st.session_state:
        st.session_state.messages = []
    
    # Tool configuration
    if "enabled_tools" not in st.session_state:
        st.session_state.enabled_tools = ["calculator", "web_search", "weather"]
    
    # Settings
    if "settings" not in st.session_state:
        st.session_state.settings = {
            "show_thinking": True,
            "show_tool_calls": True,
            "stream_response": True
        }
    
    # Analytics
    if "analytics" not in st.session_state:
        st.session_state.analytics = {
            "total_messages": 0,
            "total_tool_calls": 0,
            "session_start": datetime.now().isoformat(),
            "tool_usage": {}
        }

def get_session_stats() -> Dict[str, Any]:
    """Get current session statistics."""
    return {
        "message_count": len(st.session_state.messages),
        "user_messages": sum(1 for m in st.session_state.messages if m.get("role") == "user"),
        "assistant_messages": sum(1 for m in st.session_state.messages if m.get("role") == "assistant"),
        "enabled_tools": len(st.session_state.enabled_tools),
        "analytics": st.session_state.analytics
    }

def update_analytics(tool_calls: list):
    """Update analytics after a response."""
    st.session_state.analytics["total_messages"] += 1
    st.session_state.analytics["total_tool_calls"] += len(tool_calls)
    
    for tc in tool_calls:
        tool_name = tc.get("tool_name", "unknown")
        if tool_name not in st.session_state.analytics["tool_usage"]:
            st.session_state.analytics["tool_usage"][tool_name] = 0
        st.session_state.analytics["tool_usage"][tool_name] += 1
'''

with open(f'{app_dir}/utils/state.py', 'w') as f:
    f.write(state_py)

print("Created utils/state.py")

## Part 3: Home Page

In [ ]:
# Home.py - Main entry point
home_py = '''"""Agent Playground - Home Page"""

import streamlit as st
import sys
sys.path.insert(0, ".")

from utils.state import init_session_state, get_session_stats
from utils.agent import AVAILABLE_TOOLS

st.set_page_config(
    page_title="Agent Playground",
    page_icon="🤖",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Initialize session state
init_session_state()

# Main content
st.title("🤖 AI Agent Playground")

st.markdown("""
Welcome to the Agent Playground! This application lets you:

- **💬 Chat** with an AI agent that uses tools
- **👁️ Visualize** the agent\'s thinking process
- **🔧 Configure** which tools are available
- **📊 Analyze** session history and performance

Use the sidebar to navigate between pages.
""")

# Quick stats
st.markdown("---")
st.subheader("📈 Session Overview")

stats = get_session_stats()

col1, col2, col3, col4 = st.columns(4)

with col1:
    st.metric("💬 Messages", stats["message_count"])

with col2:
    st.metric("🔧 Tools Enabled", stats["enabled_tools"])

with col3:
    st.metric("📞 Tool Calls", stats["analytics"]["total_tool_calls"])

with col4:
    st.metric("🛠️ Available Tools", len(AVAILABLE_TOOLS))

# Available tools preview
st.markdown("---")
st.subheader("🛠️ Available Tools")

cols = st.columns(len(AVAILABLE_TOOLS))
for i, (tool_id, tool_info) in enumerate(AVAILABLE_TOOLS.items()):
    with cols[i]:
        enabled = tool_id in st.session_state.enabled_tools
        status = "✅" if enabled else "❌"
        st.markdown(f"""
        **{tool_info['icon']} {tool_info['name']}** {status}
        
        {tool_info['description']}
        """)

# Quick actions
st.markdown("---")
st.subheader("🚀 Quick Actions")

col1, col2, col3 = st.columns(3)

with col1:
    if st.button("💬 Start Chatting", use_container_width=True):
        st.switch_page("pages/1_💬_Chat.py")

with col2:
    if st.button("🔧 Configure Tools", use_container_width=True):
        st.switch_page("pages/2_🔧_Tools.py")

with col3:
    if st.button("📊 View Analytics", use_container_width=True):
        st.switch_page("pages/3_📊_Analytics.py")

# Footer
st.markdown("---")
st.caption("Built with Streamlit | Module 4.5 Demo")
'''

with open(f'{app_dir}/Home.py', 'w') as f:
    f.write(home_py)

print("Created Home.py")

## Part 4: Chat Page

In [ ]:
# pages/1_💬_Chat.py
chat_py = '''"""Agent Chat Interface"""

import streamlit as st
import json
import sys
sys.path.insert(0, ".")

from utils.state import init_session_state, update_analytics
from utils.agent import MockAgent, AVAILABLE_TOOLS

st.set_page_config(page_title="Chat", page_icon="💬", layout="wide")

# Initialize
init_session_state()

st.title("💬 Chat with Agent")

# Sidebar - Display settings
with st.sidebar:
    st.subheader("🎛️ Display Settings")
    
    st.session_state.settings["show_thinking"] = st.checkbox(
        "Show thinking process",
        value=st.session_state.settings["show_thinking"],
        help="Display the agent\'s reasoning steps"
    )
    
    st.session_state.settings["show_tool_calls"] = st.checkbox(
        "Show tool calls",
        value=st.session_state.settings["show_tool_calls"],
        help="Display tool invocations and results"
    )
    
    st.markdown("---")
    st.subheader("🔧 Active Tools")
    
    for tool_id in st.session_state.enabled_tools:
        tool = AVAILABLE_TOOLS.get(tool_id, {})
        st.write(f"{tool.get('icon', '🔧')} {tool.get('name', tool_id)}")
    
    st.markdown("---")
    if st.button("🗑️ Clear Chat", use_container_width=True):
        st.session_state.messages = []
        st.rerun()

# Main chat area
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])
        
        # Show tool calls for assistant messages
        if msg["role"] == "assistant" and st.session_state.settings["show_tool_calls"]:
            if msg.get("tool_calls"):
                with st.expander("🔧 Tool Calls", expanded=False):
                    for tc in msg["tool_calls"]:
                        tool_info = AVAILABLE_TOOLS.get(tc["tool_name"], {})
                        status = "✅" if tc["success"] else "❌"
                        st.markdown(f"**{tool_info.get('icon', '🔧')} {tc['tool_name']}** {status}")
                        st.code(json.dumps({"input": tc["input_args"], "output": tc["output"]}, indent=2), language="json")
                        st.caption(f"Duration: {tc['duration_ms']:.2f}ms")
        
        # Show thinking for assistant messages
        if msg["role"] == "assistant" and st.session_state.settings["show_thinking"]:
            if msg.get("thinking"):
                with st.expander("💭 Thinking Process", expanded=False):
                    st.markdown(msg["thinking"])

# Chat input
if prompt := st.chat_input("Ask the agent something... (try 'calculate', 'search', or 'weather')"):
    # Add user message
    st.session_state.messages.append({"role": "user", "content": prompt})
    
    with st.chat_message("user"):
        st.markdown(prompt)
    
    # Generate response
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            # Create agent with current tools
            agent = MockAgent(enabled_tools=st.session_state.enabled_tools)
            response = agent.respond(prompt)
        
        st.markdown(response.content)
        
        # Show tool calls
        if response.tool_calls and st.session_state.settings["show_tool_calls"]:
            with st.expander("🔧 Tool Calls", expanded=True):
                for tc in response.tool_calls:
                    tool_info = AVAILABLE_TOOLS.get(tc["tool_name"], {})
                    status = "✅" if tc["success"] else "❌"
                    st.markdown(f"**{tool_info.get('icon', '🔧')} {tc['tool_name']}** {status}")
                    st.code(json.dumps({"input": tc["input_args"], "output": tc["output"]}, indent=2), language="json")
                    st.caption(f"Duration: {tc['duration_ms']:.2f}ms")
        
        # Show thinking
        if response.thinking and st.session_state.settings["show_thinking"]:
            with st.expander("💭 Thinking Process", expanded=False):
                st.markdown(response.thinking)
    
    # Save to history
    st.session_state.messages.append({
        "role": "assistant",
        "content": response.content,
        "thinking": response.thinking,
        "tool_calls": response.tool_calls,
        "duration_ms": response.total_duration_ms,
        "timestamp": response.timestamp
    })
    
    # Update analytics
    update_analytics(response.tool_calls)
'''

with open(f'{app_dir}/pages/1_💬_Chat.py', 'w') as f:
    f.write(chat_py)

print("Created pages/1_💬_Chat.py")

In [ ]:
# pages/2_🔧_Tools.py
tools_py = '''"""Tool Configuration Page"""

import streamlit as st
import json
import sys
sys.path.insert(0, ".")

from utils.state import init_session_state
from utils.agent import MockAgent, AVAILABLE_TOOLS

st.set_page_config(page_title="Tools", page_icon="🔧", layout="wide")

init_session_state()

st.title("🔧 Tool Configuration")

st.markdown("""
Configure which tools the agent can use. Enable or disable tools to see how 
it affects the agent\'s responses.
""")

# Tool configuration
st.subheader("Available Tools")

cols = st.columns(2)

for i, (tool_id, tool_info) in enumerate(AVAILABLE_TOOLS.items()):
    with cols[i % 2]:
        with st.container(border=True):
            col1, col2 = st.columns([1, 4])
            
            with col1:
                st.markdown(f"# {tool_info['icon']}")
            
            with col2:
                enabled = st.checkbox(
                    tool_info["name"],
                    value=tool_id in st.session_state.enabled_tools,
                    key=f"tool_{tool_id}"
                )
                st.caption(tool_info["description"])
                
                # Update enabled tools
                if enabled and tool_id not in st.session_state.enabled_tools:
                    st.session_state.enabled_tools.append(tool_id)
                elif not enabled and tool_id in st.session_state.enabled_tools:
                    st.session_state.enabled_tools.remove(tool_id)

st.markdown("---")

# Tool testing
st.subheader("🧪 Test Tools")

st.markdown("Test individual tools to see how they work.")

selected_tool = st.selectbox(
    "Select a tool to test",
    options=list(AVAILABLE_TOOLS.keys()),
    format_func=lambda x: f"{AVAILABLE_TOOLS[x]['icon']} {AVAILABLE_TOOLS[x]['name']}"
)

if selected_tool == "calculator":
    expression = st.text_input("Enter a math expression", value="2 + 2 * 3")
    test_args = {"expression": expression}

elif selected_tool == "web_search":
    query = st.text_input("Enter a search query", value="What is Python?")
    test_args = {"query": query}

elif selected_tool == "weather":
    location = st.text_input("Enter a location", value="San Francisco")
    test_args = {"location": location}

elif selected_tool == "code_executor":
    code = st.text_area("Enter Python code", value="print(\'Hello, World!\')", height=100)
    test_args = {"code": code}

else:
    test_args = {}

if st.button("🚀 Run Test", type="primary"):
    agent = MockAgent()
    result = agent._execute_tool(selected_tool, test_args)
    
    st.subheader("Result")
    
    col1, col2 = st.columns(2)
    
    with col1:
        st.metric("Status", "✅ Success" if result.success else "❌ Failed")
    
    with col2:
        st.metric("Duration", f"{result.duration_ms:.2f} ms")
    
    st.markdown("**Output:**")
    if isinstance(result.output, (dict, list)):
        st.json(result.output)
    else:
        st.code(str(result.output))

# Current configuration
st.markdown("---")
st.subheader("📋 Current Configuration")

st.json({
    "enabled_tools": st.session_state.enabled_tools,
    "tool_count": len(st.session_state.enabled_tools)
})
'''

with open(f'{app_dir}/pages/2_🔧_Tools.py', 'w') as f:
    f.write(tools_py)

print("Created pages/2_🔧_Tools.py")

In [ ]:
# pages/3_📊_Analytics.py
analytics_py = '''"""Analytics and Session History Page"""

import streamlit as st
import pandas as pd
import json
import sys
sys.path.insert(0, ".")

from utils.state import init_session_state, get_session_stats
from utils.agent import AVAILABLE_TOOLS

st.set_page_config(page_title="Analytics", page_icon="📊", layout="wide")

init_session_state()

st.title("📊 Analytics")

stats = get_session_stats()

# Overview metrics
st.subheader("📈 Session Overview")

col1, col2, col3, col4 = st.columns(4)

with col1:
    st.metric("Total Messages", stats["message_count"])

with col2:
    st.metric("User Messages", stats["user_messages"])

with col3:
    st.metric("Agent Responses", stats["assistant_messages"])

with col4:
    st.metric("Total Tool Calls", stats["analytics"]["total_tool_calls"])

st.markdown("---")

# Tool usage chart
st.subheader("🔧 Tool Usage")

tool_usage = stats["analytics"].get("tool_usage", {})

if tool_usage:
    df = pd.DataFrame([
        {
            "Tool": AVAILABLE_TOOLS.get(tool, {}).get("name", tool),
            "Icon": AVAILABLE_TOOLS.get(tool, {}).get("icon", "🔧"),
            "Calls": count
        }
        for tool, count in tool_usage.items()
    ])
    
    col1, col2 = st.columns([2, 1])
    
    with col1:
        st.bar_chart(df.set_index("Tool")["Calls"])
    
    with col2:
        st.dataframe(df, hide_index=True)
else:
    st.info("No tool calls yet. Start chatting to generate analytics!")

st.markdown("---")

# Conversation history
st.subheader("📝 Conversation History")

if st.session_state.messages:
    for i, msg in enumerate(st.session_state.messages):
        with st.expander(f"{msg['role'].title()} ({i+1})", expanded=False):
            st.markdown(msg["content"])
            
            if msg["role"] == "assistant":
                if msg.get("duration_ms"):
                    st.caption(f"Response time: {msg['duration_ms']:.2f}ms")
                if msg.get("tool_calls"):
                    st.markdown(f"**Tool calls:** {len(msg['tool_calls'])}")
else:
    st.info("No conversation history yet.")

st.markdown("---")

# Export
st.subheader("📥 Export")

col1, col2 = st.columns(2)

with col1:
    if st.button("📄 Export as JSON", use_container_width=True):
        export_data = {
            "messages": st.session_state.messages,
            "analytics": stats["analytics"],
            "enabled_tools": st.session_state.enabled_tools
        }
        st.download_button(
            "Download JSON",
            data=json.dumps(export_data, indent=2),
            file_name="agent_session.json",
            mime="application/json"
        )

with col2:
    if st.button("📝 Export as Text", use_container_width=True):
        export_text = "Agent Playground Session\n" + "="*50 + "\n\n"
        for msg in st.session_state.messages:
            export_text += f"{msg['role'].upper()}:\n{msg['content']}\n\n"
        st.download_button(
            "Download Text",
            data=export_text,
            file_name="agent_session.txt",
            mime="text/plain"
        )
'''

with open(f'{app_dir}/pages/3_📊_Analytics.py', 'w') as f:
    f.write(analytics_py)

print("Created pages/3_📊_Analytics.py")

In [ ]:
# .streamlit/config.toml
config = '''[theme]
primaryColor = "#FF4B4B"
backgroundColor = "#FFFFFF"
secondaryBackgroundColor = "#F0F2F6"
textColor = "#262730"
font = "sans serif"

[server]
maxUploadSize = 200
'''

with open(f'{app_dir}/.streamlit/config.toml', 'w') as f:
    f.write(config)

# requirements.txt
requirements = '''streamlit>=1.30.0
pandas>=2.0.0
'''

with open(f'{app_dir}/requirements.txt', 'w') as f:
    f.write(requirements)

print("Created config and requirements files")
print(f"\n✅ Agent Playground created at: {app_dir}")
print("\nTo run: streamlit run Home.py")

---

## ✋ Lab Exercises

### Exercise 1: Add Response Time Tracking

Add a chart to the Analytics page showing response times over the session.

### Exercise 2: Add a Settings Page

Create `pages/4_⚙️_Settings.py` with options for:
- Model selection (mock dropdown)
- Temperature slider
- Max tokens

### Exercise 3: Add Streaming Responses

Modify the chat to stream the agent's response character by character.

---

## 📤 Deliverable

1. Deploy the Agent Playground to Streamlit Cloud
2. Add at least one custom tool (e.g., a translator)
3. Share the URL

---

## 🧹 Cleanup

In [ ]:
print(f"\n✅ Lab complete!")
print(f"\nFiles created in: {app_dir}")
print("\nTo run the app:")
print(f"  cd {app_dir}")
print("  streamlit run Home.py")